In [ ]:
# Setup the Jupyter version of Dash so this runs nicely inside the notebook
from jupyter_dash import JupyterDash

# Dashboard building blocks
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# OS routines so we can pull environment variables and files
import os

# Plotting and data tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import my CRUD layer for MongoDB
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Pull MongoDB connection info from env when available.
# This keeps us from hardcoding credentials into the notebook.
# Defaults are still here so the project runs in the course lab.
username = os.getenv("MONGO_USER", "aacuser")
password = os.getenv("MONGO_PASSWORD", "SNHU1234")
host = os.getenv("MONGO_HOST", "nv-desktop-services.apporto.com")
db_name = os.getenv("MONGO_DB", "AAC")

# Set up the database interface using my AnimalShelter class
shelter = AnimalShelter(host, db_name, username, password)

# Load the full dataset for the table on first load.
# If anything goes wrong, I log it and fall back to an empty frame
# so the dashboard still comes up.
try:
    df = pd.DataFrame.from_records(shelter.read({}))
    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)
except Exception as e:
    print(f"Failed to load data from MongoDB: {e}")
    df = pd.DataFrame()

#########################
# Dashboard Layout / View
#########################

# Create the Dash app instance
app = JupyterDash(__name__)

# Try to load the Grazioso logo. If it is missing,
# show a text heading instead so the UI still looks intentional.
image_filename = "Grazioso Salvare Logo.png"
try:
    encoded_image = base64.b64encode(open(image_filename, "rb").read())
    logo_img = html.Center(
        html.Img(
            src="data:image/png;base64,{}".format(encoded_image.decode()),
            height="100px",
        )
    )
except FileNotFoundError:
    logo_img = html.Center(html.H4("Grazioso Salvare Dashboard"))

# Main app layout
app.layout = html.Div(
    [
        logo_img,
        html.Center(html.H4("Created by Dominoe LaMattina")),
        html.Center(html.B(html.H1("CS 340 Dashboard"))),
        html.Hr(),

        # Simple radio filter so the user can pick a rescue type
        html.Div(
            [
                html.Label("Filter by Rescue Type"),
                dcc.RadioItems(
                    id="filter-type",
                    options=[
                        {"label": "Water Rescue", "value": "water"},
                        {"label": "Mountain or Wilderness Rescue", "value": "mountain"},
                        {"label": "Disaster or Individual Tracking", "value": "disaster"},
                        {"label": "Reset", "value": "reset"},
                    ],
                    labelStyle={"display": "inline-block", "margin-right": "15px"},
                ),
            ],
            style={"width": "80%", "margin": "auto"},
        ),

        html.Hr(),

        # DataTable shows the current view of the animals
        dash_table.DataTable(
            id="datatable-id",
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True}
                for i in df.columns
            ],
            data=df.to_dict("records"),
            row_selectable="single",
            selected_rows=[0],
            filter_action="native",
            sort_action="native",
            page_size=10,
        ),

        html.Br(),
        html.Hr(),

        # Bottom row holds the chart and the map next to each other
        html.Div(
            className="row",
            style={"display": "flex", "justify-content": "space-evenly"},
            children=[
                html.Div(id="graph-id", className="col s12 m6", style={"width": "48%"}),
                html.Div(id="map-id", className="col s12 m6", style={"width": "48%"}),
            ],
        ),
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################

def rescue_filter_query(filter_type):
    """
    Build a MongoDB query based on the selected rescue type.
    I keep this logic in one place so it is easy to tweak or extend.
    """
    if filter_type == "water":
        return {
            "breed": {
                "$in": [
                    "Labrador Retriever Mix",
                    "Chesapeake Bay Retriever",
                    "Newfoundland",
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    elif filter_type == "mountain":
        return {
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler",
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    elif filter_type == "disaster":
        return {
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler",
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        }
    else:
        # Reset or no selection shows everything
        return {}


# Update DataTable based on rescue type filter
@app.callback(
    Output("datatable-id", "data"),
    [Input("filter-type", "value")],
)
def update_dashboard(filter_type):
    """
    When the user picks a rescue type, rebuild the table data
    by querying MongoDB with the matching filter.
    """
    query = rescue_filter_query(filter_type)
    filtered_df = pd.DataFrame.from_records(shelter.read(query))

    if "_id" in filtered_df.columns:
        filtered_df.drop(columns=["_id"], inplace=True)

    return filtered_df.to_dict("records")


# Update the breed distribution pie chart based on the current table view
@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_virtual_data")],
)
def update_graphs(viewData):
    """
    Take whatever data the table is currently showing
    and build a simple pie chart to visualize breed distribution.
    """
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    # Guard in case breed is missing in the current view
    if "breed" not in dff.columns:
        return []

    fig = px.pie(dff, names="breed", title="Breed Distribution")

    return [dcc.Graph(figure=fig)]


# Highlight selected columns in the table so the user gets some feedback
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")],
)
def update_styles(selected_columns):
    """
    Simple visual cue for which columns have been clicked.
    I only style the columns the user actually selects.
    """
    if not selected_columns:
        return []

    return [
        {
            "if": {"column_id": i},
            "background_color": "#D2F3FF",
        }
        for i in selected_columns
    ]


# Update the map based on the selected row in the table
@app.callback(
    Output("map-id", "children"),
    [
        Input("datatable-id", "derived_virtual_data"),
        Input("datatable-id", "derived_virtual_selected_rows"),
    ],
)
def update_map(viewData, index):
    """
    When a user selects a row in the table,
    center the map marker on that animal's location.
    """
    if viewData is None or index is None or len(index) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    # Bail out early if we do not have location data in the current view
    if "location_lat" not in dff.columns or "location_long" not in dff.columns:
        return []

    lat = dff.iloc[row]["location_lat"]
    lon = dff.iloc[row]["location_long"]
    breed = dff.iloc[row].get("breed", "Unknown Breed")
    name = dff.iloc[row].get("name", "Unknown Name")

    return [
        dl.Map(
            style={"width": "100%", "height": "500px"},
            center=[lat, lon],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup(
                            [
                                html.H4("Animal Name"),
                                html.P(name),
                            ]
                        ),
                    ],
                ),
            ],
        )
    ]


# Launch the app inside Jupyter
# debug=True is fine for this course project, but I would turn it off in production.
app.run_server(debug=True)

Dash app running on http://127.0.0.1:17941/
